In [34]:
from langchain_community.tools import (
    StructuredTool,
    ShellTool,
    DuckDuckGoSearchRun,
    WikipediaQueryRun,
    RequestsGetTool,
    RequestsPostTool,
    ReadFileTool,
    WriteFileTool,
    ListDirectoryTool,
    BaseTool,
)


In [20]:
from langchain_core.tools import tool

In [26]:
import json
from langchain_core.messages import AIMessage, HumanMessage
# from langchain_huggingface import  ChatHuggingFace,HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from huggingface_hub import inference
from langchain_openai import ChatOpenAI
from dotenv import  load_dotenv
load_dotenv()

True

In [ ]:
# llms=HuggingFaceEndpoint(
#     repo_id="deepseek-ai/DeepSeek-V3.2",
#     task="text-generation"
# )
# model=ChatHuggingFace(llm=llms)

In [30]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# or, depending on the specific library version:
# model = ChatGoogleGenerativeAI(model_name="gemini-2.5-flash")


1. Define a tool using the @tool decorator

In [31]:
@tool
def get_current_weather(city: str)->str:
    """Get the current weather in a given city."""
    if city == "San Francisco":
        return json.dumps({"city": city, "weather": "sunny", "temperature": "70F"})
    else:
        return json.dumps({"city": city, "weather": "unknown", "temperature": "unknown"})

In [32]:
# model = ChatOpenAI(model="gpt-4o", temperature=0)
model_with_tools = model.bind_tools([get_current_weather])

In [36]:
prompt = "What is the weather in San Francisco?"
ai_message = model_with_tools.invoke([HumanMessage(content=prompt)])
# 4. Process the AIMessage to access the tool requests
if ai_message.tool_calls:
    print("Model requested the following tool calls:")
    for tool_call in ai_message.tool_calls:
        print(f"* Tool ID: {tool_call['id']}")
        print(f"* Tool Name: {tool_call['name']}")
        print(f"* Tool Args: {tool_call['args']}")
        # You would then typically execute the tool using these arguments
        print(f"  -> Executing {tool_call['name']}...")
        result = get_current_weather.func(**tool_call['args']) 
        print(f" -> Result: {result}")
else:
    print("Model did not request any tools.")

Model requested the following tool calls:
* Tool ID: 2118192c-7d57-4c36-bb11-9b75444cc059
* Tool Name: get_current_weather
* Tool Args: {'city': 'San Francisco'}
  -> Executing get_current_weather...
 -> Result: {"city": "San Francisco", "weather": "sunny", "temperature": "70F"}
